In [15]:
import music21 as m2
import glob
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from os import path
from scipy.signal import windows, oaconvolve
%config Completer.use_jedi=False

In [5]:
import sys

code_path = path.abspath(path.join('..','..','diss3_code','midi'))
print(code_path, path.exists(code_path))

sys.path.append(code_path)
import midi_utils as mu

/home/siegman/Documents/Shahar/BarIlan/Dissertation3/code/diss3_code/midi True


In [21]:
from importlib import reload
reload(mu);

In [4]:
midi_paths = ['MIDI-Freestyle', 'MIDI-Interaction']
midi_paths = [path.abspath(path.join('..','..','..','data',p)) for p in midi_paths]
print([path.exists(p) for p in midi_paths])
midi_files = sum([glob.glob(path.join(mp,'*.mid')) for mp in midi_paths],[] )

[True, True]


In [7]:
all_ticks = mu.populate_music_dictionary(midi_files)

('1009', 'Freestyle', 43)
('1009', 'Freestyle', 45)
('1009', 'Freestyle', 48)
('1025', 'Freestyle', 43)
('1025', 'Freestyle', 48)
('1025', 'Freestyle', 45)
('1021', 'Freestyle', 45)
('1021', 'Freestyle', 48)
('1021', 'Freestyle', 43)
('1018', 'Freestyle', 43)
('1018', 'Freestyle', 48)
('1018', 'Freestyle', 45)
('1008', 'Freestyle', 48)
('1008', 'Freestyle', 45)
('1008', 'Freestyle', 43)
('1048', 'Freestyle', 45)
('1048', 'Freestyle', 43)
('1048', 'Freestyle', 48)
('1026', 'Freestyle', 43)
('1026', 'Freestyle', 45)
('1026', 'Freestyle', 48)
('1024', 'Freestyle', 43)
('1024', 'Freestyle', 48)
('1024', 'Freestyle', 45)
('1023', 'Freestyle', 48)
('1023', 'Freestyle', 43)
('1023', 'Freestyle', 45)
('1033', 'Freestyle', 45)
('1033', 'Freestyle', 43)
('1033', 'Freestyle', 48)
('1045', 'Freestyle', 48)
('1045', 'Freestyle', 43)
('1045', 'Freestyle', 45)
('1034', 'Freestyle', 45)
('1034', 'Freestyle', 48)
('1034', 'Freestyle', 43)
('1046', 'Freestyle', 48)
('1046', 'Freestyle', 45)
('1046', 'Fr

In [8]:
ticks_per_sec = {(1003, 'non-predictable'): 176,
 (1005, 'non-predictable'): 176,
 (1008, 'non-predictable'): 208,
 (1010, 'non-predictable'): 176,
 (1012, 'non-predictable'): 176,
 (1014, 'non-predictable'): 176,
 (1016, 'non-predictable'): 192,
 (1018, 'non-predictable'): 192,
 (1020, 'non-predictable'): 176,
 (1022, 'non-predictable'): 176,
 (1024, 'non-predictable'): 176,
 (1026, 'non-predictable'): 176,
 (1028, 'non-predictable'): 176,
 (1030, 'non-predictable'): 176,
 (1032, 'non-predictable'): 176,
 (1034, 'non-predictable'): 176,
 (1036, 'non-predictable'): 176,
 (1003, 'freestyle'): 96,
 (1005, 'freestyle'): 128,
 (1008, 'freestyle'): 128,
 (1010, 'freestyle'): 128,
 (1012, 'freestyle'): 128,
 (1014, 'freestyle'): 128,
 (1016, 'freestyle'): 96,
 (1018, 'freestyle'): 128,
 (1020, 'freestyle'): 128,
 (1022, 'freestyle'): 128,
 (1024, 'freestyle'): 128,
 (1026, 'freestyle'): 128,
 (1028, 'freestyle'): 128,
 (1030, 'freestyle'): 128,
 (1032, 'freestyle'): 96,
 (1034, 'freestyle'): 96,
 (1036, 'freestyle'): 96}

In [13]:
ticks_per_sec_complete = {(gr,session): ticks_per_sec.get((int(gr),'freestyle' if session == 'Freestyle' else 'non-predictable'), 96) for gr, session, _ in all_ticks.keys()}

In [14]:
ticks_per_sec_complete

{('1009', 'Freestyle'): 96,
 ('1025', 'Freestyle'): 96,
 ('1021', 'Freestyle'): 96,
 ('1018', 'Freestyle'): 128,
 ('1008', 'Freestyle'): 128,
 ('1048', 'Freestyle'): 96,
 ('1026', 'Freestyle'): 128,
 ('1024', 'Freestyle'): 128,
 ('1023', 'Freestyle'): 96,
 ('1033', 'Freestyle'): 96,
 ('1045', 'Freestyle'): 96,
 ('1034', 'Freestyle'): 96,
 ('1046', 'Freestyle'): 96,
 ('1035', 'Freestyle'): 96,
 ('1030', 'Freestyle'): 128,
 ('1031', 'Freestyle'): 96,
 ('1032', 'Freestyle'): 96,
 ('1014', 'Freestyle'): 128,
 ('1002', 'Freestyle'): 96,
 ('1016', 'Freestyle'): 96,
 ('1042', 'Freestyle'): 96,
 ('1013', 'Freestyle'): 96,
 ('1029', 'Freestyle'): 96,
 ('1047', 'Freestyle'): 96,
 ('1050', 'Freestyle'): 96,
 ('1049', 'Freestyle'): 96,
 ('1003', 'Freestyle'): 96,
 ('1004', 'Freestyle'): 96,
 ('1038', 'Freestyle'): 96,
 ('1022', 'Freestyle'): 128,
 ('1015', 'Freestyle'): 96,
 ('1007', 'Freestyle'): 96,
 ('1051', 'Freestyle'): 96,
 ('1044', 'Freestyle'): 96,
 ('1041', 'Freestyle'): 96,
 ('1019', 'Fr

In [43]:
def resample(drum_signals, ticks_per_sec, window_size_secs, window_parameter_secs):
    kernel = windows.gaussian(window_size_secs*ticks_per_sec, window_parameter_secs*ticks_per_sec)
    kernel = kernel/kernel.sum()
    y_resampled = []
    for signal in drum_signals:
        x = mu.to_dense(np.array(signal))
        y = oaconvolve(x, kernel)
        sampling_interval = int(ticks_per_sec/4) # for a 4 Hz sampling frequency
        y_resampled.append(y[::sampling_interval]*sampling_interval)
        
    max_vec_len = max([len(v) for v in y_resampled])
    max_vec_len = max_vec_len+ (4-(max_vec_len % 4) if (max_vec_len % 4) else 0)
    y_resampled = [np.hstack([v, [0.]*(max_vec_len-len(v))]) for v in y_resampled]
    ret = [np.vstack([np.arange(0,int(max_vec_len/4),0.25),v]) for v in y_resampled]
    
    return ret


In [44]:
all_ticks_time_base = [{ 
    'group': gr, 
    'session': sess, 
    'signals': resample(
        [all_ticks[gr,sess, dr] for dr in [43, 45, 48]], 
        ticks_per_sec_complete[gr,sess], 
        20, 
        2
    ) 
    } for gr, sess in set((k[0], k[1]) for k in all_ticks.keys())
]

In [48]:
all_ticks_time_base_expanded = sum([[{
    'group': data['group'], 
    'session': data['session'],
    'drummer': dr,
    'signal': data['signals'][i]
} for i, dr in enumerate([43,45,48])] for data in all_ticks_time_base],[])

In [51]:
midi_df = pd.DataFrame(all_ticks_time_base_expanded)
midi_df.head()

,group,session,drummer,signal
0,1014,Freestyle,43,"[[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, ..."
1,1014,Freestyle,45,"[[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, ..."
2,1014,Freestyle,48,"[[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, ..."
3,1047,Interaction,43,"[[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, ..."
4,1047,Interaction,45,"[[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, ..."


In [52]:
midi_df.shape

(300, 4)

In [53]:
data_dir = path.abspath('../../../data')
data_dir, path.exists(data_dir)

('/home/siegman/Documents/Shahar/BarIlan/Dissertation3/data', True)

In [54]:
midi_df.to_json(path.join(data_dir, 'midi_time_based.json'))